In [2]:
import pysam
import glob
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
title = "Histgram of 1083 Insert Region Reads " + key_ID
def Histplot_Save(_plot_data, _title, _path):
    # Example code to create and save the histogram
    plt.figure(figsize=(10, 6))
    ax = sns.histplot(data = _plot_data, x = "Length", hue='Type', element='step', 
         log_scale=(False, True), bins=region_end - region_start)
    # Add a title
    ax.set_title(_title)
    output_file = _path.replace(".bam", "_hist.png")
    plt.savefig(output_file, dpi=100, bbox_inches='tight')
    plt.close()
    return None

def Generate_INDEL_STAT_For_Alignment_In_Given_Region(_bam_path, _chr, _start, _end):
    """
    only using sequences inside certain region, and return Insert, delete & Match for each alignment
    """
    # Iterate through each read in the BAM file
    bam_file = pysam.AlignmentFile(_bam_path, "rb")
    chromosome = _chr
    region_start = _start
    region_end = _end

    # Initialize counters
    match_lengths = []
    insertion_lengths = []
    deletion_lengths = []
    sub_lengths = []
    query_names = []
    
    for read in bam_file.fetch():
        match_length = 0
        insertion_length = 0
        deletion_length = 0
        sub_length = 0
        # Skip unmapped reads
        if read.is_unmapped:
            continue
    
        query_names.append(read.qname)
        # Initialize total lengths for the current read
        total_match_length = 0
        total_insertion_length = 0
        total_deletion_length = 0
    
        # Current genomic position as the read is aligned
        read_genomic_pos = read.reference_start
    
        # Process CIGAR tuples to aggregate INDELs and match lengths for the read
        for cigar_tuple in read.cigartuples:
            operation = cigar_tuple[0]
            length = cigar_tuple[1]
    
            # Calculate the next genomic position based on the current CIGAR operation
            if operation in [0, 7, 8]:  # Match, sequence match, sequence mismatch (M, =, X)
                region_overlap_start = max(read_genomic_pos, region_start)
                region_overlap_end = min(read_genomic_pos + length, region_end)
    
                # If the CIGAR operation overlaps the region of interest
                if region_overlap_start < region_overlap_end:
                    # Add to the total match length for this read
                    total_match_length += region_overlap_end - region_overlap_start
                read_genomic_pos += length
    
            elif operation == 1:  # Insertion
                # Count the insertion if it occurs within the region
                if region_start <= read_genomic_pos < region_end:
                    total_insertion_length += length
    
            elif operation == 2:  # Deletion
                region_overlap_start = max(read_genomic_pos, region_start)
                region_overlap_end = min(read_genomic_pos + length, region_end)
    
                # If the deletion overlaps the region
                if region_overlap_start < region_overlap_end:
                    total_deletion_length += region_overlap_end - region_overlap_start
                read_genomic_pos += length
    
            elif operation in [3, 4, 5]:  # Skips, soft clips, hard clips (N, S, H)
                if operation == 3:  # Skip (N) also moves the genomic position
                    read_genomic_pos += length
    
        # Store the aggregated lengths for the current read
        if total_match_length >= 0:
            match_lengths.append(total_match_length)
        if total_insertion_length >= 0:
            insertion_lengths.append(total_insertion_length)
        if total_deletion_length >= 0:
            deletion_lengths.append(total_deletion_length)# Close the BAM file
    bam_file.close()
    
    match_data = {
        'Type': ['Match'] * len(match_lengths) + ['Ins'] * len(insertion_lengths) + 
        ['Del'] * len(deletion_lengths), # + ['Sub'] * len(sub_lengths),
        'Length': match_lengths + insertion_lengths + deletion_lengths# + sub_lengths
    }
    
    df_CIGAR_sum = pd.DataFrame(data = {"qname": query_names, "Match":match_lengths, "Ins":insertion_lengths,
                        "Del": deletion_lengths})
    return match_data, df_CIGAR_sum

In [10]:
bam_files = sorted(glob.glob("*.bam"))

['1083_Overlap_AAVS1_edited_1.bam',
 '1083_Overlap_AAVS1_edited_2.bam',
 '1083_Overlap_AAVS1_edited_3.bam',
 '1083_Overlap_AAVS1_edited_4.bam']

In [34]:
input_bam_path = "1083_Overlap_Reads_AAVS1_edited_4.bam"

_path_excel = "1083_insert_INDEL_Analysis.xlsx"
with pd.ExcelWriter(_path_excel) as writer:
    for input_bam_path in bam_files:
        key_ID = input_bam_path.replace(".bam", "").replace("1083_Overlap_", "")
        if ("AAVS1" in key_ID):
            chromosome, region_start, region_end = "AAVS1_edited_1", 221, 1304
        elif("PRNP" in key_ID):
            chromosome, region_start, region_end = "PRNP_edited", 185, 1268
        elif("HEK3" in key_ID):
            chromosome, region_start, region_end = "HEK3_edited", 136, 1219
        elif("IDS" in key_ID):
            chromosome, region_start, region_end = "IDS_edited", 258, 1341
        
        plot_data, df_STAT = Generate_INDEL_STAT_For_Alignment_In_Given_Region(input_bam_path, chromosome, region_start, region_end)
        df_STAT.to_excel(writer, sheet_name = key_ID, index=None,
                         freeze_panes=(1, 1))
        Histplot_Save(plot_data, "Histgram of 1083 Insert Region Reads " + key_ID, input_bam_path)